In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cvx

In [16]:
equity_list = ['A', 'B']
alpha_1 = pd.Series(index=equity_list, data=[2.4, 2.5])
alpha_2 = pd.Series(index=equity_list, data=[2.5, 2.4])
benchmark_weight = pd.Series(index=equity_list, data=[0.5, 0.5])

sigma_A, sigma_B = 0.42, 0.33
rho_AB = 0.7
cov_mat = [
    [sigma_A ** 2, rho_AB * sigma_A * sigma_B],
    [rho_AB * sigma_A * sigma_B, sigma_B ** 2]
]
cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=cov_mat)
print(alpha_1, alpha_2, cov_df)

A    2.4
B    2.5
dtype: float64 A    2.5
B    2.4
dtype: float64          A        B
A  0.17640  0.09702
B  0.09702  0.10890


In [17]:
# alpha和cov扩展ｃａｓｈ
alpha_1['cash'] = 0.
alpha_2['cash'] = 0.
cov_df = cov_df.reindex(equity_list + ['cash']).fillna(0.)
cov_df['cash'] = 0.
trading_day = '20100101'
cov_panel = pd.Panel(
    data={trading_day: cov_df})
alpha_1_df = pd.DataFrame(
    index=[trading_day],
    data=[alpha_1])
alpha_2_df = pd.DataFrame(
    index=[trading_day],
    data=[alpha_2])

# benchmark
benchmark_weight['cash'] = 0.

/home/casey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
from solar.cvxportfolio.returns import ReturnsForecast
from solar.solver.components.risks import AssetSigma
from solar.cvxportfolio.returns import ReturnsForecast
from solar.cvxportfolio.constraints import LongCash, LongOnly
from solar.solver.components.policies import MaxReturnPolicy
from solar.solver.components.constraints import AssetWeightBound, GroupWeightBound, MaxRiskLimit
from solar.math.omega_math import calc_volatility_by_V
solver_param = {}
solver_param.update({
    cvx.ECOS: {
        'abstol': 1.0e-09,
        'reltol': 1.0e-09,
        'max_iters': 150
    }
})

In [22]:
class LongOnlyRHACSolver(object):
    def __init__(self, param):
        self.trading_day = param['trading_day']
        self.asset_list = param['asset_list']
        self.benchmark_weight_series = param['benchmark_weight_series']
        self.alpha_series = param['alpha_series']
        self.asset_cov = param['asset_cov']
        self.asset_order = self.alpha_series.index.tolist()
        self.alpha_df = pd.DataFrame(index=[self.trading_day], data=[param['alpha_series']])
        self.asset_cov_panel = pd.Panel(data={self.trading_day: self.asset_cov})
        self.total_weight = param['total_weight']
        self.asset_lower_boundary_series = param['asset_lower_boundary_series']
        self.asset_upper_boundary_series = param['asset_upper_boundary_series']
        self.target_risk = param['target_risk']
        self.pre_holding = param['pre_holding']
        self.current_price = param['current_price_series']
        self.halt_stocks = self.current_price[self.current_price == 0.0].index.tolist()
        self.current_price.replace(0., 100000, inplace=True)
        self.solver = param['solver']

    def solve_without_round(self):
        result = self._solve_opt_frac_trade()
        return result

    def _solve_opt_frac_trade(self):
        alpha_mdl = ReturnsForecast(self.alpha_df)
        asset_cov_mdl = AssetSigma(self.asset_cov_panel)
        risk_cons = MaxRiskLimit(max_risk=self.target_risk, risk_model=asset_cov_mdl,
                                 w_bench=self.benchmark_weight_series)

        asset_group_bound = []

        weight_bound_list = [x for x in zip(self.asset_lower_boundary_series, self.asset_upper_boundary_series)]

        if self.total_weight:
            # weight_bound_list += [1 - self.total_weight]
            asset_group_bound.append(
                {'index': [i for i in range(len(self.asset_list))], 'cb': self.total_weight}
            )
        group_wt_cons = GroupWeightBound(asset_group_bound)
        wt_cons = AssetWeightBound(weight_bound_list)

        constraints = [LongOnly(), risk_cons, wt_cons, group_wt_cons]

        policy = MaxReturnPolicy(
            alpha_mdl, costs=[],
            constraints=constraints,
            solver=self.solver)

        prob, trade_value = policy.get_trades(self.pre_holding, t=self.trading_day)

        trade_volume = trade_value / self.current_price
        trade_volume.fillna(0., inplace=True)
        after_trade_value = self.pre_holding + trade_value
        after_trade_volume = after_trade_value / self.current_price
        after_trade_volume.fillna(0., inplace=True)

        after_trade_ratio = after_trade_value / after_trade_value.sum()
        exante_risk = np.sqrt(
            calc_volatility_by_V(self.asset_cov, after_trade_ratio))
        exante_return = self.alpha_series.dot(after_trade_ratio)
        benchmark_risk = np.sqrt(
            calc_volatility_by_V(self.asset_cov, self.benchmark_weight_series))
        active_weight = after_trade_ratio - self.benchmark_weight_series
        exante_active_risk = np.sqrt(
            calc_volatility_by_V(self.asset_cov, active_weight))
        trade_result = {
            'trade_value': trade_value, 'trade_volume': trade_volume,
            'expected_weight': after_trade_ratio,
            'expected_holding': after_trade_value,
            'expected_volume': after_trade_volume, 'opt_status': prob.status, 'expected_active_weight': active_weight,
            'exante_risk': exante_risk, 'exante_return': exante_return,
            'benchmark_risk': benchmark_risk,
            'exante_active_risk': exante_active_risk
        }

        return trade_result


In [27]:
asset_lower_boundary_series = pd.Series(index=equity_list, data=-1)
asset_upper_boundary_series = pd.Series(index=equity_list, data=1.)
# asset_group_lower_boundary_list = []
# asset_group_upper_boundary_list = []
pre_holding_series = pd.Series(index=equity_list, data=0.)
pre_holding_series['cash'] = 1.
market_price_series = pd.Series(index=equity_list, data=1.)
market_price_series['cash'] = 1.
target_risk = 0.10

In [28]:
param = {
    'trading_day': trading_day, 
    'asset_list': equity_list,
    'benchmark_weight_series': benchmark_weight,
    'alpha_series': alpha_1,
    'asset_cov': cov_df,
    'asset_lower_boundary_series': asset_lower_boundary_series,
    'asset_upper_boundary_series': asset_upper_boundary_series,
    'pre_holding': pre_holding_series,
    'current_price_series': market_price_series,
    'target_risk': target_risk, 
    'solver': 'ECOS',
    'total_weight': 1.
}

mdl = LongOnlyRHACSolver(param)
mdl.solve_without_round()

(0.0, Expression(AFFINE, UNKNOWN, (3,)))


/home/casey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  app.launch_new_instance()


{'trade_value': A       0.168976
 B       0.831024
 cash   -1.000000
 dtype: float64, 'trade_volume': A       0.168976
 B       0.831024
 cash   -1.000000
 dtype: float64, 'expected_weight': A       0.168976
 B       0.831024
 cash    0.000000
 dtype: float64, 'expected_holding': A       0.168976
 B       0.831024
 cash    0.000000
 dtype: float64, 'expected_volume': A       0.168976
 B       0.831024
 cash    0.000000
 dtype: float64, 'opt_status': 'optimal', 'expected_active_weight': A      -0.331024
 B       0.331024
 cash    0.000000
 dtype: float64, 'exante_risk': 0.32785799566696716, 'exante_return': 2.483102421587079, 'benchmark_risk': 0.3461719226049392, 'exante_active_risk': 0.09999999947019851}

In [26]:
import QuantLib as ql